In [1]:
import pandas as pd

In [2]:
weather = pd.read_csv('delta_weather.csv')

In [3]:
Delta = pd.read_csv('delta_air_lines_flights_test.csv')

In [4]:
weather['Name'].unique()

array(['Atlanta, GA, United States', 'Charlotte, NC, United States',
       'Chicago, IL, United States', 'Denver, CO, United States',
       'Fort Worth Ave, Dallas, TX 75208, United States',
       'Houston, TX, United States', 'Los Angeles, CA, United States',
       'New York, NY, United States', 'Seattle, WA, United States',
       'Washington, DC, United States'], dtype=object)

In [5]:
my_lambda = lambda x: ''.join(x.split(',')[0]).lower()
weather['Name'] = weather['Name'].apply(my_lambda)

In [6]:
weather['Name'].replace('fort worth ave', 'dallas/fort worth', inplace=True)

In [7]:
weather['Name'].unique()

array(['atlanta', 'charlotte', 'chicago', 'denver', 'dallas/fort worth',
       'houston', 'los angeles', 'new york', 'seattle', 'washington'],
      dtype=object)

In [8]:
weather['Date time'] = pd.to_datetime(weather['Date time'])
weather['Date time'] = weather['Date time'].dt.strftime('%Y-%m-%d')

In [9]:
weather_wvc = weather[['Name', 'Date time', 'Wind Speed', 'Visibility', 'Conditions']]

### Delta

In [10]:
Delta.head()

,fl_date,mkt_carrier,origin_airport_id,origin_city_name,dest_airport_id,dest_city_name,crs_dep_time,crs_arr_time,crs_elapsed_time,distance
0,2020-01-31,DL,11057,"Charlotte, NC",12953,"New York, NY",1229,1430,121,544
1,2020-01-31,DL,10397,"Atlanta, GA",12953,"New York, NY",1630,1846,136,762
2,2020-01-31,DL,12953,"New York, NY",10397,"Atlanta, GA",1900,2148,168,762
3,2020-01-31,DL,13930,"Chicago, IL",12953,"New York, NY",1545,1911,146,733
4,2020-01-31,DL,12953,"New York, NY",10397,"Atlanta, GA",2000,2242,162,762


In [11]:
Delta['origin_city_name'] = Delta['origin_city_name'].apply(my_lambda)
Delta['dest_city_name'] = Delta['dest_city_name'].apply(my_lambda)

In [12]:
### Adding columns based on origin city name
Delta_data = pd.merge(Delta, weather_wvc, how = 'left', left_on=['origin_city_name', 'fl_date'], right_on=['Name', 'Date time']).rename(columns={"Wind Speed": "origin_wind_speed",
                                                                                                                                            "Visibility": "origin_visibility",
                                                                                                                                            "Conditions": "origin_conditions"})

In [13]:
### Adding columns based on dest city name
Delta_data = pd.merge(Delta_data, weather_wvc, how = 'left', left_on=['dest_city_name', 'fl_date'], right_on=['Name', 'Date time']).rename(columns={"Wind Speed": "dest_wind_speed",
                                                                                                                                            "Visibility": "dest_visibility",
                                                                                                                                            "Conditions": "dest_conditions"})

In [14]:
Delta_data.tail()

,fl_date,mkt_carrier,origin_airport_id,origin_city_name,dest_airport_id,dest_city_name,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,Name_x,Date time_x,origin_wind_speed,origin_visibility,origin_conditions,Name_y,Date time_y,dest_wind_speed,dest_visibility,dest_conditions
12048,2020-01-01,DL,11278,washington,12953,new york,1600,1730,90,214,washington,2020-01-01,14.7,9.9,Partially cloudy,new york,2020-01-01,15.1,9.9,Overcast
12049,2020-01-01,DL,12953,new york,11057,charlotte,1630,1849,139,544,new york,2020-01-01,15.1,9.9,Overcast,charlotte,2020-01-01,8.6,9.9,Partially cloudy
12050,2020-01-01,DL,12953,new york,11278,washington,1000,1129,89,214,new york,2020-01-01,15.1,9.9,Overcast,washington,2020-01-01,14.7,9.9,Partially cloudy
12051,2020-01-01,DL,12478,new york,13930,chicago,1125,1319,174,740,new york,2020-01-01,15.1,9.9,Overcast,chicago,2020-01-01,19.6,9.9,"Snow, Partially cloudy"
12052,2020-01-01,DL,12953,new york,11278,washington,1900,2022,82,214,new york,2020-01-01,15.1,9.9,Overcast,washington,2020-01-01,14.7,9.9,Partially cloudy


In [15]:
Delta_data = Delta_data.drop(['Name_x', 'Date time_x', 'Name_y', 'Date time_y'], axis = 1)

In [16]:
Delta_data.columns

Index(['fl_date', 'mkt_carrier', 'origin_airport_id', 'origin_city_name',
       'dest_airport_id', 'dest_city_name', 'crs_dep_time', 'crs_arr_time',
       'crs_elapsed_time', 'distance', 'origin_wind_speed',
       'origin_visibility', 'origin_conditions', 'dest_wind_speed',
       'dest_visibility', 'dest_conditions'],
      dtype='object')

In [17]:
Delta_data['day_of_the_week'] = pd.to_datetime(Delta_data['fl_date']).dt.day_name()

In [18]:
Delta_data['fl_date'] = pd.to_datetime(Delta_data['fl_date'])

In [19]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
cal = calendar()

In [20]:
holidays = cal.holidays(start='2020-01-01', end='2020-01-31')
Delta_data['holiday'] = Delta_data['fl_date'].isin(holidays)

In [21]:
Delta_data.head()

,fl_date,mkt_carrier,origin_airport_id,origin_city_name,dest_airport_id,dest_city_name,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,origin_wind_speed,origin_visibility,origin_conditions,dest_wind_speed,dest_visibility,dest_conditions,day_of_the_week,holiday
0,2020-01-31,DL,11057,charlotte,12953,new york,1229,1430,121,544,NaN,NaN,NaN,NaN,NaN,NaN,Friday,False
1,2020-01-31,DL,10397,atlanta,12953,new york,1630,1846,136,762,NaN,NaN,NaN,NaN,NaN,NaN,Friday,False
2,2020-01-31,DL,12953,new york,10397,atlanta,1900,2148,168,762,NaN,NaN,NaN,NaN,NaN,NaN,Friday,False
3,2020-01-31,DL,13930,chicago,12953,new york,1545,1911,146,733,NaN,NaN,NaN,NaN,NaN,NaN,Friday,False
4,2020-01-31,DL,12953,new york,10397,atlanta,2000,2242,162,762,NaN,NaN,NaN,NaN,NaN,NaN,Friday,False


In [22]:
Delta_data.to_csv('Delta_data.csv', sep=",", index = False)